<a href="https://colab.research.google.com/github/Derinhelm/llm_tokenization/blob/main/token_aligning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Derinhelm/llm_tokenization

Cloning into 'llm_tokenization'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 16 (delta 6), reused 14 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 22.25 MiB | 10.64 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
import sys
sys.path.append('/content/llm_tokenization')

In [4]:
from data_classes import ConllEntry, Sentence

In [5]:
import pickle

with open('/content/llm_tokenization/treebank_test_sets/treebank_data.pickle', 'rb') as f:
    treebanks = pickle.load(f)

In [6]:
treebank_names = list(treebanks.keys())
treebank_names

['gsd', 'pud', 'syntagrus', 'poetry', 'taiga']

In [7]:
import pickle

with open('/content/llm_tokenization/results/sent_subtokens.pickle', 'rb') as f:
    original_sent_subtokens = pickle.load(f)

In [8]:
model_names = list(original_sent_subtokens.keys())

In [9]:
model_names

['Qwen3_4B', 'RuadaptQwen3_4B_Instruct']

In [10]:
sent_subtokens = { m_name: {} for m_name in model_names }
for m_name in model_names:
  sent_subtokens[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_i in original_sent_subtokens[m_name][tr].keys():
      d = original_sent_subtokens[m_name][tr][sent_i]
      sent_subtokens[m_name][tr][sent_i] = {}
      sent_subtokens[m_name][tr][sent_i]['token_texts'] = d[1][3:-2]
      sent_subtokens[m_name][tr][sent_i]['offsets'] = d[0]['offset_mapping'][0][3:-2] - d[0]['offset_mapping'][0][3][0]

In [117]:
tokens = { m_name: {} for m_name in model_names }
tokens_dict = { m_name: {} for m_name in model_names }
for m_name in model_names:
  tokens[m_name] = { tr: {} for tr in treebank_names }
  tokens_dict[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_i, sent_info in sent_subtokens[m_name][tr].items():
      s_tokens = []
      cur_subtokens = []
      s_tokens_dict = {}
      cur_s = ''
      for subt in sent_info['token_texts']:
        if subt != ' ' and ((subt.isalpha() and subt[0] != ' ' and cur_s.isalpha()) or (subt.isdigit() and subt.isdigit())):
          cur_s += subt
          cur_subtokens.append(subt)
        else:
          cur_s = cur_s.strip()
          s_tokens.append(cur_s)
          if cur_s:
            if cur_s not in s_tokens_dict:
              s_tokens_dict[cur_s] = [cur_subtokens]
            else:
              s_tokens_dict[cur_s].append(cur_subtokens)
          subt = subt[:].strip()
          cur_s = subt
          if subt:
            cur_subtokens = [subt]
          else:
            cur_subtokens = []
      if cur_s:
        s_tokens.append(cur_s.strip())
        s_tokens_dict[cur_s] = [ cur_subtokens ]
      s_tokens = [s for s in s_tokens if s]
      tokens[m_name][tr][sent_i] = s_tokens
      tokens_dict[m_name][tr][sent_i] = s_tokens_dict

In [118]:
tokens['Qwen3_4B']['gsd']['test-s2']

['Стоимость',
 'проезда',
 'с',
 '5',
 'января',
 '2013',
 'года',
 '--',
 '15',
 'рублей',
 ',',
 'движение',
 'осуществляется',
 'с',
 '6',
 '.00',
 'до',
 '00',
 '.20',
 '.']

In [119]:
tokens_dict['Qwen3_4B']['gsd']['test-s2']

{'Стоимость': [['С', 'то', 'им', 'ость']],
 'проезда': [['про', 'ез', 'да']],
 'с': [['с'], ['с']],
 '5': [['5']],
 'января': [['января']],
 '2013': [['2', '0', '1', '3']],
 'года': [['года']],
 '--': [['--']],
 '15': [['1', '5']],
 'рублей': [['рублей']],
 ',': [[',']],
 'движение': [['движ', 'ение']],
 'осуществляется': [['осуществля', 'ется']],
 '6': [['6']],
 '.00': [['.', '0', '0']],
 'до': [['до']],
 '00': [['0', '0']],
 '.20': [['.', '2', '0']],
 '.': [['.']]}

# Сравнение частотности количества подтокенов

In [122]:
from collections import Counter

for tr in treebank_names:
  for m_name in model_names:
    print(m_name, tr)
    print(Counter([ len(sub_list) for s_td in tokens_dict[m_name][tr].values() for sub_lists in s_td.values() for sub_list in sub_lists ]).most_common())
  print()

Qwen3_4B gsd
[(1, 5571), (3, 2117), (2, 1772), (4, 1448), (5, 550), (6, 182), (7, 49), (8, 14), (9, 6), (10, 2)]
RuadaptQwen3_4B_Instruct gsd
[(1, 8416), (2, 1624), (3, 1126), (4, 437), (5, 86), (6, 16), (9, 2), (10, 1)]

Qwen3_4B pud
[(1, 10304), (2, 3114), (3, 2969), (4, 2115), (5, 802), (6, 231), (7, 60), (8, 16), (9, 6), (10, 1)]
RuadaptQwen3_4B_Instruct pud
[(1, 15601), (2, 2314), (3, 1096), (4, 428), (5, 77), (6, 8)]

Qwen3_4B syntagrus
[(1, 86041), (2, 25651), (3, 23898), (4, 15023), (5, 5998), (6, 1653), (7, 387), (8, 96), (9, 20), (10, 7), (12, 1), (11, 1)]
RuadaptQwen3_4B_Instruct syntagrus
[(1, 126366), (2, 19417), (3, 9264), (4, 2836), (5, 429), (6, 94), (7, 10)]

Qwen3_4B poetry
[(1, 5204), (3, 2029), (2, 1643), (4, 926), (5, 283), (6, 65), (7, 13), (8, 9), (10, 4), (9, 3), (11, 1)]
RuadaptQwen3_4B_Instruct poetry
[(1, 7315), (2, 1711), (3, 927), (4, 197), (5, 22), (6, 3), (7, 2), (8, 1)]

Qwen3_4B taiga
[(1, 5771), (2, 1731), (3, 1547), (4, 976), (5, 339), (6, 96), (7, 16

In [123]:
token_subtokens = { m_name: {} for m_name in model_names }
for m_name in model_names:
  token_subtokens[m_name] = { tr: {} for tr in treebank_names }
  for tr in treebank_names:
    for sent_i, t, sub_list in [ (sent_i, t, sub_list) for sent_i, s_td in tokens_dict[m_name][tr].items() for t, sub_lists in s_td.items() for sub_list in sub_lists ]:
        if t not in token_subtokens[m_name][tr]:
          token_subtokens[m_name][tr][t] = [(sent_i, sub_list)]
        else:
          token_subtokens[m_name][tr][t].append((sent_i, sub_list))

In [139]:
ambiguous_subtokens = { m_name: {} for m_name in model_names }
for m_name in model_names:
  ambiguous_subtokens[m_name] = { tr: [] for tr in treebank_names }
  for tr in treebank_names:
    for t, st_list in token_subtokens[m_name][tr].items():
        diff_subtokens = set(map(lambda x: str(x[1]), st_list))
        if len(diff_subtokens) != 1:
          ambiguous_subtokens[m_name][tr].append((t, st_list))

    print(m_name, tr, len(ambiguous_subtokens[m_name][tr]))


Qwen3_4B gsd 17
Qwen3_4B pud 37
Qwen3_4B syntagrus 450
Qwen3_4B poetry 44
Qwen3_4B taiga 47
RuadaptQwen3_4B_Instruct gsd 21
RuadaptQwen3_4B_Instruct pud 41
RuadaptQwen3_4B_Instruct syntagrus 576
RuadaptQwen3_4B_Instruct poetry 33
RuadaptQwen3_4B_Instruct taiga 35


In [134]:
m_name, tr

('RuadaptQwen3_4B_Instruct', 'taiga')

In [136]:
print(sent_subtokens['RuadaptQwen3_4B_Instruct']['taiga']['4015']['token_texts']) # В-ам
print(sent_subtokens['RuadaptQwen3_4B_Instruct']['taiga']['4041']['token_texts']) #Вам

['В', 'сев', 'олод', ',', 'Вы', ' же', ' лицо', ' "', 'Граждан', 'ской', ' Сил', 'ы', '",', 'В', 'ам', ' что', ' стыдно', ' быть', ' похожим', ' на', ' депутата', ' главенств', 'ующей', ' партии', ' РФ', 'Ст', 'ыд', 'но', '!']
['В', 'я', 'чес', 'лав', ',', ' почему', ' бы', ' Вам', ' не', ' возглавить', ' КПРФ', ' Пока', ' оно', ' е', 'щ', 'ё', ' есть', '.']


In [138]:
print(sent_subtokens['RuadaptQwen3_4B_Instruct']['taiga']['2127']['token_texts']) # Солнце
print(sent_subtokens['RuadaptQwen3_4B_Instruct']['taiga']['2129']['token_texts']) # С-олн-це

['*', ' *', ' *', ' Солнце', ' ль', 'ет', ' жар', 'кий', ' мед', ' в', ' степи', ' и', ' луга', ',', ' Пл', 'атья', ' жел', 'той', ' пар', 'чи', ' дар', 'ит', ' всем', ' подряд', '.']
['С', 'олн', 'це', ' см', 'уг', 'лой', ' рукой', ' травы', ' в', 'орош', 'ит', ',', ' В', ' зол', 'очен', 'ой', ' печи', ' яблоки', ' печ', 'ет', ',', ' Каждый', ' лист', ' на', ' вет', 'в', 'ях', ' золотом', ' прош', 'ит', ',', ' И', ' река', ' золотой', ' лав', 'ою', ' течет', '.']


In [142]:
print(sent_subtokens['Qwen3_4B']['taiga']['6009']['token_texts']) # Да
print(sent_subtokens['Qwen3_4B']['taiga']['6011']['token_texts']) # Д-а

['--', ' Да', ' все', '!']
['Д', 'а', ' же', ' пед', 'али', ' су', '..', 'ки', ' от', 'п', 'или', 'ли', '!!!']


# Сравнение наборов субтокенов для одних и тех же токенов

# Сравнение разбиений на токены

In [215]:
diff_tokens = {tr: [] for tr in treebank_names}
m_name1, m_name2 = model_names[:2]
for tr in treebank_names:
  for s in treebanks[tr]:
    if tokens[m_name1][tr][s.sent_id] != tokens[m_name2][tr][s.sent_id]:
      diff_tokens[tr].append(s)
    #print(s.sent_id)
    #print(tokens[m_name1][tr][s.sent_id], "\n", tokens[m_name2][tr][s.sent_id], "\n")
  print(tr, len(diff_tokens[tr]))

gsd 22
pud 43
syntagrus 198
poetry 42
taiga 35


In [216]:
from collections import Counter

for tr in treebank_names:
  for s in diff_tokens[tr]:
    s_id = s.sent_id
    print(tr, s.text)
    tokens1 = Counter(tokens[m_name1][tr][s_id])
    tokens2 = Counter(tokens[m_name2][tr][s_id])
    print(tokens1 - tokens2, tokens2 - tokens1)
    print()

gsd Узнав о её существовании на допросе благовещенского диакона Федора Иванова, правительство озаботилось её конфискацией, со всеми подготовленными списками, и поручило написать на неё опровержение газскому митр.
Counter({'оз': 1, 'а�': 1, '�от': 1, 'илось': 1}) Counter({'озаботилось': 1})

gsd На юго-западе села живописный пруд.
Counter({'�': 2, 'го': 1}) Counter({'юго': 1})

gsd В июле 1962 года американская противоракета ``Найк-Зевс&#39;&#39; впервые смогла поразить боевой блок МБР ``Атлас&#39;&#39; (первый такой перехват в истории был осуществлён 4 марта 1961 года в Сары-Шагане ракетой В-1000), однако ввиду недостаточного технического совершенства и дороговизны так и не была принята на вооружение, хотя многие американские ЗРК были позже разработаны с применением технологий проекта ``Найк&#39;&#39;.
Counter({'ра�': 1, '�': 1, 'етой': 1}) Counter({'ракетой': 1})

gsd Работа была прервана.
Counter({'Р': 1, 'а�': 1, '�от': 1, 'а': 1}) Counter({'Работа': 1})

gsd На рисунке показан прин

In [217]:
diff_gold_tokens1 = {tr: [] for tr in treebank_names}
diff_gold_tokens2 = {tr: [] for tr in treebank_names}

for tr in treebank_names:
  for s in treebanks[tr]:
    gold_tokens = [t.form for t in s.tokens if '.' not in t.id]
    if tokens[m_name1][tr][s.sent_id] != gold_tokens:
      diff_gold_tokens1[tr].append(s)
    if tokens[m_name2][tr][s.sent_id] != gold_tokens:
      diff_gold_tokens2[tr].append(s)
    #print(s.sent_id)
    #print(tokens[m_name1][tr][s.sent_id], "\n", tokens[m_name2][tr][s.sent_id], "\n")
  print(tr, len(diff_tokens[tr]), len(diff_gold_tokens1[tr]), len(diff_gold_tokens2[tr]))

gsd 22 225 215
pud 43 279 252
syntagrus 198 2234 2115
poetry 42 101 99
taiga 35 243 241


In [218]:
from collections import Counter

for tr in treebank_names:
  for s in diff_gold_tokens1[tr]:
    s_id = s.sent_id
    print(tr, s_id)
    print(s.text)
    tokens1 = Counter(tokens[m_name1][tr][s_id])
    tokens2 = Counter([t.form for t in s.tokens if '.' not in t.id])
    print(tokens1 - tokens2, tokens2 - tokens1)
    print()

Streaming output truncated to the last 5000 lines.
syntagrus 2010Korrektsiya_mifov.xml_19
Майкл Макфол и Кэтрин Стоунер-Вайс из Стэнфордского университета попытались в своей статье подвести итоги правления второго президента России, как Шлейфер и Трейсман в свое время подводили итоги правления Бориса Ельцина и первых двух лет Владимира Путина.
Counter({'Стоунер': 1, '-': 1, 'Вайс': 1}) Counter({'Стоунер-Вайс': 1})

syntagrus 2010Korrektsiya_mifov.xml_21
По их мнению, нынешние темпы роста российского ВВП довольно высоки, но, во-первых, обеспечены высокими ценами на сырье, а во-вторых, могли быть куда выше.
Counter({'во': 2, '-': 2, 'первых': 1, 'вторых': 1}) Counter({'во-первых': 1, 'во-вторых': 1})

syntagrus 2010Korrektsiya_mifov.xml_22
"Остается только гадать, насколько быстрее бы росла российская экономика при более демократичной системе", - говорит Макфол, который считает правление Путина скорее провалом, чем успехом для России.
Counter({'",': 1}) Counter({'"': 1, ',': 1})

syntagr

In [219]:
from collections import Counter

for tr in treebank_names:
  for s in diff_gold_tokens2[tr]:
    s_id = s.sent_id
    print(tr, s_id)
    print(s.text)
    tokens1 = Counter(tokens[m_name2][tr][s_id])
    tokens2 = Counter([t.form for t in s.tokens if '.' not in t.id])
    print(tokens1 - tokens2, tokens2 - tokens1)
    print()

gsd test-s1
Билли начал играть за резервный состав ``Черка&#39;&#39; в возрасте 16 лет, а через пару сезонов был приглашён в основной состав.
Counter({'&#39': 1, ';&#39': 1, ';': 1}) Counter({'&#39;&#39;': 1})

gsd test-s2
Стоимость проезда с 5 января 2013 года -- 15 рублей, движение осуществляется с 6.00 до 00.20.
Counter({'6': 1, '.00': 1, '00': 1, '.20': 1}) Counter({'6.00': 1, '00.20': 1})

gsd test-s3
Стал членом секретариата общественной безопасности Мексики (SSP, Secretaría de Seguridad Pública) и специальным уполномоченным Федеральной полиции Мексики (PFP, Policía Federal Preventiva).
Counter({').': 1}) Counter({')': 1, '.': 1})

gsd test-s7
В военные годы одним из самых активных заказчиков Нарбута стал литературно-художественный и сатирический журнал ``Лукоморье&#39;&#39;.
Counter({'литературно': 1, '-': 1, 'художественный': 1, '&#39': 1, ';&#39': 1, ';.': 1}) Counter({'литературно-художественный': 1, '&#39;&#39;': 1, '.': 1})

gsd test-s9
Если эффект оказался недостаточным, п

In [149]:
"Ю".encode("utf-8")

b'\xd0\xae'

In [160]:
a = sent_subtokens['Qwen3_4B']['gsd']['test-s95']['token_texts'][1] + sent_subtokens['Qwen3_4B']['gsd']['test-s95']['token_texts'][2]
a = a.strip()
a

'��'

In [165]:
ord(a[0]), ord(a[1])

(65533, 65533)

In [162]:
a.encode('latin1').decode('utf-8', errors='replace')

AttributeError: 'str' object has no attribute 'decode'

In [156]:
sent_subtokens['Qwen3_4B']['gsd']['test-s95']['token_texts'][1].encode('utf-8') + sent_subtokens['Qwen3_4B']['gsd']['test-s95']['token_texts'][2].encode('utf-8')

b' \xef\xbf\xbd\xef\xbf\xbd'

In [ ]:
sent_subtokens['Qwen3_4B']['gsd']['test-s95']

In [145]:
hex(ord(sent_subtokens['Qwen3_4B']['gsd']['test-s95']['token_texts'][2]))

'0xfffd'

In [140]:
tr = 'gsd'
s_id = diff_tokens[tr][1].sent_id
diff_tokens[tr][1].text, s_id

('На юго-западе села живописный пруд.', 'test-s95')

In [133]:
m_name1, m_name2

('Qwen3_4B', 'RuadaptQwen3_4B_Instruct')

In [130]:
tokens[m_name1][tr][s_id]

['На', '�', '�го', '-западе', 'села', 'живописный', 'пруд', '.']

In [131]:
tokens[m_name2][tr][s_id]

['На', 'юго', '-западе', 'села', 'живописный', 'пруд', '.']